In [48]:
import cv2
import numpy as np
import os

In [49]:
root= os.path.abspath(os.path.join(os.getcwd(), '..'))

In [79]:
#input
input_video_path  = os.path.join(root, 'data/input_video.mp4')
ad_image_path_1 = os.path.join(root, 'data/ad1.png')
ad_image_path_2 = os.path.join(root, 'data/ad2.png')
output_video_path = os.path.join(root, 'outputs/ad_overlay.mp4')

In [86]:
# Load ad image and resize
ad_image_1 = cv2.imread(ad_image_path_1, cv2.IMREAD_UNCHANGED)

if ad_image_1.shape[2] == 4:
    ad_rgb = ad_image_1[:, :, :3]
    ad_alpha = ad_image_1[:, :, 3]
else:
    ad_rgb = ad_image_1
    ad_alpha = np.ones(ad_rgb.shape[:2], dtype=np.uint8) * 255

ad_rgb = cv2.resize(ad_rgb, (450, 300))
ad_alpha = cv2.resize(ad_alpha, (450, 300))

In [69]:
# Load ad image and resize
ad_image_2 = cv2.imread(ad_image_path_2, cv2.IMREAD_UNCHANGED)

if ad_image_2.shape[2] == 4:
    ad_rgb = ad_image_2[:, :, :3]
    ad_alpha = ad_image_2[:, :, 3]
else:
    ad_rgb = ad_image_2
    ad_alpha = np.ones(ad_rgb.shape[:2], dtype=np.uint8) * 255

ad_rgb = cv2.resize(ad_rgb, (450, 300))
ad_alpha = cv2.resize(ad_alpha, (450, 300))

In [87]:
src_pts = np.array([
    [0, 0],
    [ad_rgb.shape[1], 0],
    [ad_rgb.shape[1], ad_rgb.shape[0]],
    [0, ad_rgb.shape[0]]
], dtype=np.float32)

In [88]:
# === VIDEO SETUP ===
cap = cv2.VideoCapture(input_video_path)
ret, first_frame = cap.read()

if not ret:
    print("Error reading video.")
    cap.release()
    exit()

first_frame = cv2.resize(first_frame, (1280, 720))
frame_size = (first_frame.shape[1], first_frame.shape[0])
fps = cap.get(cv2.CAP_PROP_FPS)


In [89]:
# === SELECT FIELD CORNERS ===
print("Select 4 points on the field for the ad placement.")
selected_points = []

def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN and len(selected_points) < 4:
        selected_points.append((x, y))
        print(f"Point {len(selected_points)}: ({x}, {y})")

clone = first_frame.copy()
cv2.namedWindow("Select Field Points")
cv2.setMouseCallback("Select Field Points", mouse_callback)

while len(selected_points) < 4:
    temp = clone.copy()
    for pt in selected_points:
        cv2.circle(temp, pt, 5, (0, 255, 0), -1)
    cv2.imshow("Select Field Points", temp)
    cv2.waitKey(1)

cv2.destroyWindow("Select Field Points")
dst_pts = np.array(selected_points, dtype=np.float32)

Select 4 points on the field for the ad placement.
Point 1: (60, 336)
Point 2: (297, 276)
Point 3: (523, 312)
Point 4: (316, 381)


In [90]:
# === VIDEO WRITER ===
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

In [91]:
# === MAIN LOOP ===
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, frame_size)

    # Homography transformation
    H = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped_ad = cv2.warpPerspective(ad_rgb, H, frame_size)
    warped_alpha = cv2.warpPerspective(ad_alpha, H, frame_size)

    mask = warped_alpha / 255.0
    inv_mask = 1.0 - mask

    for c in range(3):
        frame[:, :, c] = frame[:, :, c] * inv_mask + warped_ad[:, :, c] * mask

    frame = frame.astype(np.uint8)
    out.write(frame)
    cv2.imshow("Virtual Ad Overlay", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

